# खर्च दाबी विश्लेषण

यो नोटबुकले कसरी प्लगइनहरू प्रयोग गरेर स्थानीय रसिद छविबाट यात्रा खर्चहरू प्रक्रिया गर्ने, खर्च दाबी इमेल तयार गर्ने, र पाई चार्ट प्रयोग गरेर खर्च डेटा दृश्यात्मक गर्ने एजेन्टहरू बनाउने देखाउँछ। एजेन्टहरूले कार्यको सन्दर्भ अनुसार गतिशील रूपमा फङ्सनहरू चयन गर्छन्।

चरणहरू:
1. OCR एजेन्टले स्थानीय रसिद छविलाई प्रक्रिया गरेर यात्रा खर्च डेटा निकाल्छ।
2. इमेल एजेन्टले खर्च दाबी इमेल तयार गर्छ।

### यात्रा खर्चको उदाहरण:
कल्पना गर्नुहोस् कि तपाईं अर्को शहरमा व्यापार बैठकको लागि यात्रा गरिरहेको कर्मचारी हुनुहुन्छ। तपाईंको कम्पनीले सबै उचित यात्रा सम्बन्धित खर्चहरूको क्षतिपूर्ति दिने नीति राखेको छ। यहाँ सम्भावित यात्रा खर्चहरूको विवरण छ:
- यातायात:
तपाईंको घर शहरबाट गन्तव्य शहरसम्मको राउन्ड ट्रिपको हवाई टिकट।
एयरपोर्टसम्म र एयरपोर्टबाट ट्याक्सी वा राइड-हेलिङ सेवाहरू।
गन्तव्य शहरमा स्थानीय यातायात (जस्तै सार्वजनिक यातायात, भाडामा लिने कार, वा ट्याक्सी)।

- आवास:
बैठक स्थल नजिकैको मध्यम स्तरको व्यापार होटलमा तीन रातको बसाइ।

- भोजन:
कम्पनीको दैनिक भत्ता नीतिमा आधारित ब्रेकफास्ट, लन्च, र डिनरको दैनिक भोजन भत्ता।

- विविध खर्चहरू:
एयरपोर्टमा पार्किङ शुल्क।
होटलमा इन्टरनेट पहुँच शुल्क।
टिप्स वा साना सेवा शुल्कहरू।

- कागजात:
तपाईंले सबै रसिदहरू (उडान, ट्याक्सी, होटल, भोजन आदि) र क्षतिपूर्ति प्राप्त गर्न पूरा गरिएको खर्च रिपोर्ट पेश गर्नुहुन्छ।


## आवश्यक पुस्तकालयहरू आयात गर्नुहोस्

नोटबुकका लागि आवश्यक पुस्तकालयहरू र मोड्युलहरू आयात गर्नुहोस्।


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## खर्च मोडेलहरू परिभाषित गर्नुहोस्

 व्यक्तिगत खर्चहरूको लागि Pydantic मोडेल बनाउनुहोस् र प्रयोगकर्ताको सोधलाई संरचित खर्च डाटामा परिवर्तन गर्नको लागि एक ExpenseFormatter कक्षा तयार गर्नुहोस्।

 प्रत्येक खर्च निम्न स्वरूपमा प्रस्तुत गरिनेछ:
 `{'date': '07-Mar-2025', 'description': 'गन्तव्यमा उडान', 'amount': 675.99, 'category': 'यातायात'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## एजेन्ट परिभाषित गर्दै - इमेल तयार पार्दै

खर्च दाबी पेश गर्न इमेल तयार पार्नको लागि एउटा एजेन्ट क्लास बनाउनुहोस्।  
- यो एजेन्टले `kernel_function` डेकोरेटर प्रयोग गरेर खर्च दाबी पेश गर्न इमेल तयार पार्ने एउटा फङ्सन परिभाषित गर्छ।  
- यसले खर्चहरूको कुल रकम गणना गर्छ र विवरणलाई इमेलको सामग्रीमा ढाल्छ।  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# रसिद छविहरूबाट यात्रा खर्च निकाल्ने एजेन्ट

रसिद छविहरूबाट यात्रा खर्च निकाल्नको लागि एजेन्ट क्लास बनाउनुहोस्।  
- यो एजेन्टले `kernel_function` डेकोरेटर प्रयोग गरेर यात्रा खर्च निकाल्ने कार्य परिभाषित गर्दछ।  
- रसिद छविलाई OCR (Optical Character Recognition) प्रयोग गरेर पाठमा रूपान्तरण गर्नुहोस् र मिति, विवरण, रकम, र श्रेणीजस्ता सम्बन्धित जानकारी निकाल्नुहोस्।  


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## खर्चहरू प्रक्रिया गर्दै

खर्चहरू प्रक्रिया गर्न आवश्यक एजेन्टहरू सिर्जना र दर्ता गरेर त्यसलाई सक्रिय गर्ने एक असिंक्रोनस कार्य परिभाषित गर्नुहोस्।
- यो कार्यले खर्चहरू प्रक्रिया गर्दा वातावरण चरहरू लोड गर्दछ, आवश्यक एजेन्टहरू सिर्जना गर्दछ, र तिनीहरूलाई प्लगइनको रूपमा दर्ता गर्दछ।
- यसले दुई एजेन्टहरूसँग समूह च्याट सिर्जना गर्दछ र खर्चहरूको डाटाको आधारमा इमेल र पाई चार्ट उत्पन्न गर्न एक प्रम्प्ट सन्देश पठाउँछ।
- यसले च्याट सक्रिय गर्दा हुने कुनै पनि त्रुटिहरूलाई व्यवस्थापन गर्दछ र एजेन्टहरूको उचित सफाइ सुनिश्चित गर्दछ।


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## मुख्य फङ्सन

मुख्य फङ्सन परिभाषित गरी कन्सोल खाली गर्नुहोस् र `process_expenses` फङ्सनलाई असिन्क्रोनस रूपमा चलाउनुहोस्।


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
